In [2]:
import pandas as pd
import pandas as pd
import random
import boto3
import random
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
import joblib
import gzip
import json
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
import pickle
from keras.preprocessing.text import Tokenizer
import collections
from collections import Counter
import re as regex
from scipy.sparse import csr_matrix
import os
import warnings
warnings.filterwarnings('ignore')
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
# from gensim import utils
from bs4 import BeautifulSoup as BSHTML
from gensim.parsing.preprocessing import preprocess_string, STOPWORDS #, remove_stopword_tokens
import boto3
import logging
# from ppring import ppring
from botocore.exceptions import ClientError
import requests
from PIL import Image
import sys # to access the system
import cv2
from IPython.display import Image 

### M2 LAMBDA DEPENDENCIES

In [3]:
# INPUT FILES:
"datasets/M2_tfidf_mtx"     
'models/M2_tfidf.joblib'
'index/M2_category_labels.data'
'datasets/M2_rank.data'
'index/M2_category_index.pickle'

# OUTPUT: top 5 category labels and index

'index/M2_category_index.pickle'

In [4]:
''' Lambda function: takes a query and returns the top 5 category keys of recommendations
INPUT files: "M2_tfidf_mtx.csv" , "M2.joblib" ,  "M2_categories.json" + query
OUTPUT: top 5 tuples of matching categoris
1- Creates a class tfidf and download dependencies in the first instance, tries to save in cache
2- Takes each query and vectorize
3- Calculate simirality tuples
4- Ranks recommendation based on similarity and np.log(count of asin in each category) '''

# match=set(self.M2.get_feature_names_out()) & set(liste)
# creating class to import necessary matrices and models for retrival
class tfidf:
    def __init__(self):
        # load cleaned group_by data sets and models from disk
        self.mtx=self.load_sparse_csr("datasets/M2_tfidf_mtx")        
        self.M2 = joblib.load('models/M2_tfidf.joblib')
        self.stemmer = PorterStemmer() 
        self.quer_R1 = None
        self.simi = None
        
        with open('index/M2R1_big_dico.pickle', 'rb') as filehandle:
            self.big_index = pickle.load(filehandle)
        
        self.tfidf_mtx = self.load_sparse_csr("datasets/R1_tfidf_mtx")
        self.model = joblib.load('models/R1_tfidf.joblib')
            
        with open('datasets/M2_rank.data', 'rb') as filehandle:
            self.rank = np.array(pickle.load(filehandle))
            
        with open('index/M2_category_index.pickle', 'rb') as filehandle:
            self.category_index = pickle.load(filehandle)
            
        with open('index/M2R1_asin_category.pickle', 'rb') as filehandle:
            self.asin_category = pickle.load(filehandle)

            
    def load_sparse_csr(self, filename):
        # here we need to add .npz extension manually
        loader = np.load(filename + '.npz')
        return csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                      shape=loader['shape']) 
    
    def asin_of_categories(self,cat_idx):
        
        info = self.big_index[cat_idx]
        cat = info[0]
        start = info[1]
        finish = info[2]

        self.simi = cosine_similarity(self.tfidf_mtx,self.quer_R1)
        simi_mtx = self.simi[start:finish,]
            
        arr2 = np.argsort(-simi_mtx,axis=0)[:5] + start
        dim = arr2.shape[0]
        idx_asin = list(arr2.reshape(dim,)) 
        
        return idx_asin
        
        
    
    ## retrieve match tfidf (LAMBDA FUNCTION)
    def M2_query_to_category_function(self,q):
        stop_words= set(stopwords.words('english'))
        liste=[self.stemmer.stem(w) for w in word_tokenize(q) if not w.lower() in stop_words]
        n_valid_words= len([x for x in liste if len(x)>2])
        string=' '.join(element for element in liste)       
        quer = self.M2.transform([string])
        self.quer_R1 = self.model.transform([string])
        
        simi = cosine_similarity(self.mtx,quer) 
        
        Beta_factor = 0.0
        simi_mtx = simi + self.rank * Beta_factor # rank is a coeff that you can tweak np.log(gb["count"] + 1) * 0.35
        
        # results = list(self.category[np.argsort(-simi_mtx,axis=0)[:5].reshape(5,)])
        idx= list(np.argsort(-simi_mtx,axis=0)[:5].reshape(5,))
        
        liste = []
        for element in idx:
            ##### M2 returns indices
            retour = self.asin_of_categories(element)
            liste = liste + retour
        
        return idx, np.round(np.max(simi),2), n_valid_words, liste
    
        

In [5]:
%%time
# take as input the 1000 query test data frame and returns the metrics in an array
def testing_queries(min_query,max_query, query_test,threshold_list): 
    # calling the class
    ob = tfidf() 
    spread = max_query-min_query
    arr = np.zeros((spread * len(threshold_list)*2, 13))
    for n_words in range(0,spread):    
        valid = 0
        col = "query_{}".format(n_words + min_query)
        queries = list(zip(query_test[col], query_test.cat_idx, query_test.label, query_test.asin))

        for element in queries:

            q = element[0]
            cat_idx = element[1]
            label = element[2]
            asin_idx = element[3]

            liste_idx, first_maxi, n_valid_words, liste_asin_first = ob.M2_query_to_category_function(q)
            valid += n_valid_words

            # for matches in range(1,len(idx)):
            first = liste_idx #[:matches]
            first_match = len(set(first) & set([cat_idx]))

            matches_asin = len(set([asin_idx]) & set(liste_asin_first))

            for top_n in range(0,2):
                if top_n == 1:
                    first = liste_idx[:3]
                    first_match = len(set(first) & set([cat_idx]))
                    
                for j, threshold in enumerate(threshold_list):
                    indice = j + n_words * len(threshold_list) + top_n * (spread * len(threshold_list))
                    # if label == 1
                    if label:
                        if (first_maxi > threshold and first_match) :
                            # counting TP and matches
                            arr[indice,0] += 1
                            arr[indice,1] += 1
                            ### at asin level
                            arr[indice,9] += matches_asin
                        else:
                            # counting FN
                            arr[indice,3] += 1
                            arr[indice,10] += 1
                    # if label == 0
                    else:
                        # counting FP and matches
                        if (first_maxi > threshold):
                            arr[indice,0] += 1
                            arr[indice,2] += 1
                            arr[indice,11] += 1
                        # counting TN 
                        else:
                            arr[indice,4] += first_match
                            arr[indice,12] += 1
                    # counting real stem words
                    arr[indice,5] = np.round(valid / len(queries),2)
                    arr[indice,6] = threshold
                    arr[indice,7] = n_words + min_query
                    arr[indice,8] = len(first)
    return arr

# takes the array of metrics and returns the formated data frame with all the metrics and the merged
def formating_array(arr, filename, longueur):  
    col=["match","TP","FP","FN","TN","stem_words","threshold","n_words","TOP_N","TP_asin","FN_asin", "FP_asin","TN_asin"]  
    df = pd.DataFrame(arr, columns = col)
    df["queries"] = longueur
    df["TN"] = longueur - (df["TP"] + df["FP"] + df["FN"])
    df= df [['n_words','stem_words',"threshold","TOP_N",'queries','match', 'TP','FP', 'FN', 'TN',"TP_asin","FN_asin", "FP_asin","TN_asin"]]
    df["accuracy"] = np.round((df["TP"] + df["TN"]) / longueur, 2) # check
    df["Precision"] = np.round(df["TP"]/ (df["TP"] + df["FP"]),2) # check
    df["Recall"] = np.round(df["TP"]/ (df["TP"] + df["FN"]), 2) #check

    df["FN_asin"] = df["FN_asin"] + df["TP_asin"] - df["TP_asin"]
    df["accuracy_asin"] = np.round((df["TP_asin"] + df["TN_asin"]) / longueur, 2) # check
    df["Precision_asin"] = np.round(df["TP_asin"]/ (df["TP_asin"] + df["FP_asin"]),2) # check
    df["Recall_asin"] = np.round(df["TP_asin"]/ (df["TP_asin"] + df["FN_asin"]), 2) 
    df["F1"] = np.round((2 * df["Precision"] * df["Recall"]) / (df["Precision"] + df["Recall"]),2)
    df["F1_asin"] = np.round((2 * df["Precision_asin"] * df["Recall_asin"]) / (df["Precision_asin"] + df["Recall_asin"]),2)
    
    # export to csv file
    df1 = df.reset_index()
    df1 = df1[["index","threshold","accuracy","Precision","Recall","F1","F1_asin"]]
    df1.columns = ["idx","threshold","accuracy","Precision","Recall","F1","F1_asin"]
    df.to_csv(filename, index=False)
    
    return df 

# Takes a data frame and returns 
def top_results(metrics,data):
    df1 = data.reset_index()
    df1 = df1[["index","threshold","accuracy","Precision","Recall","F1","Precision_asin","F1_asin"]]
    df1.columns = ["idx","threshold","accuracy","Precision","Recall","F1","Precision_asin","F1_asin"]
    
    col = metrics
    machin = data.groupby(['n_words','TOP_N'])[col].nlargest(1).reset_index()
    del machin[col]
    machin.columns = ["n_words","top_n","idx"]
    df_merge = machin.merge(df1, on = "idx", suffixes=(False, False))
    df_merge["n_words"] = df_merge["n_words"].astype(int)
    df_merge["top_n"] = df_merge["top_n"].astype(int)
    del df_merge["idx"]
    
    return df_merge


def initialize():
    # range of length of query words to test [start, finish[ 
    min_query = 3
    max_query = 8

    ############# KEEP TRUE AS DEFAULT ##############
    download = True
    ############################################
    
    # will dowload the filename results
    if download:
        filename = "results/M2R1_full_original_results.csv" 
        df_results = pd.read_csv(filename)  
    
    # re-runs everything based on importing the query_test dataset
    if not download:       
        query_test = pd.read_csv("datasets/query_test_1000.csv") 
        # Setting the similarity threshold range
        threshold_list = [x/100 for x in range(0,101)]
        # threshold_list = [0.30,0.45,0.50]
        # query_test = query_test.sample(100)
        ## returns the array with all the metrics, longuest query
        arr = testing_queries(min_query,max_query, query_test,threshold_list)
        longueur = len(query_test)
        df_results = formating_array(arr, filename, longueur)  
        filename1 = "results/M2R1_full_original_results.csv"
        df_results.to_csv(filename1, index = False)

    ### ANALYSIS FIELD
    # returning top results for a specific metrics
    metrics = "F1"
    summary_results = top_results(metrics, df_results)
    filename2 = "results/M2R1_original_summary_results.csv"
    summary_results.to_csv(filename2, index = False)
    
    return df_results, summary_results


CPU times: total: 0 ns
Wall time: 0 ns


In [6]:
%time
df_results, summary_results = initialize()

CPU times: total: 0 ns
Wall time: 0 ns


In [15]:
df_results

In [7]:
summary_results

,n_words,top_n,threshold,accuracy,Precision,Recall,F1,Precision_asin,F1_asin
0,3,3,0.00,0.29,0.19,0.13,0.15,0.11,0.09
1,3,5,0.00,0.31,0.24,0.17,0.20,0.15,0.13
2,4,3,0.00,0.27,0.20,0.16,0.18,0.15,0.13
3,4,5,0.21,0.32,0.25,0.19,0.22,0.20,0.17
4,5,3,0.24,0.32,0.25,0.19,0.22,0.21,0.18
5,5,5,0.18,0.27,0.26,0.24,0.25,0.21,0.20
6,6,3,0.22,0.29,0.25,0.21,0.23,0.22,0.20
7,6,5,0.22,0.31,0.28,0.24,0.26,0.25,0.23
8,7,3,0.22,0.30,0.26,0.22,0.24,0.25,0.23
9,7,5,0.23,0.32,0.29,0.25,0.27,0.28,0.25
